<a href="https://colab.research.google.com/github/Nada24/create-JSON-Files/blob/main/Finale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip3 install python-docx
!pip install docx
!pip install spacy
!python -m spacy download fr_core_news_sm

from docx import Document
import pandas as pd
import numpy as np
import nltk
import re
import unicodedata
import string
import fr_core_news_sm
import numpy as np
import json
from nltk.corpus import stopwords
nlp = fr_core_news_sm.load()
nltk.download('stopwords')

def cleaning(test):
   test=test.lower()
   test = test.translate(str.maketrans('','',string.punctuation))
   test=test.strip()
   test=''.join( c for c in test if  c not in "’'–" )
   return test

def return_token(sentence):
    doc = nlp(sentence)
    return [X.text for X in doc]

def remove_stopw(test):
    stopWords = set(stopwords.words('french'))
    clean_words = ''
    for token in return_token(test):
        if token not in stopWords:
            clean_words = clean_words + token + ' '
    return clean_words

def lemm(test):
  tokens = [] 
  test3 = nlp(test)
  for token in test3: 
     tokens.append(token) 
  
  lemmatized_sentence = " ".join([token.lemma_ for token in test3]) 
  return lemmatized_sentence

def accentfree(test):
  test2 = ''.join((c for c in unicodedata.normalize('NFD', test) if unicodedata.category(c) != 'Mn'))
  return test2

def normalize(test):
  test=cleaning(test)
  test=remove_stopw(test)
  test=lemm(test)
  test = accentfree(test)
  test=' '.join(test.split())
  return test

#************************ Initial Conditions ********************************
def Initcond (ch) :   
   lista=[]
   i="Initial Conditions"
   dictt = { i :[]}
   listax= ch.splitlines( )
   for line in listax:
       if re.search(r'«', line):
          x = normalize(line[:line.find('«')])
          y = accentfree(line[line.find('«')+1:line.rfind('»')].replace('\u00a0',''))
       elif re.search(r'LP', line):
          x = normalize(line[:line.find('LP')])
          y = line[line.find('LP'):-1].replace('\u00a0','')
       elif re.search(r'Diagbox', line): 
           x = normalize(line[line.find('Diagbox'):])
           y = ''
       elif re.search(r'DiagBox', line): 
           x = normalize(line[line.find('DiagBox'):])
           y = ''
       elif re.search(r'Ecran affiché', line) :
           x = normalize(line[:line.find('choix')]).replace('\u00a0','')
           y = line[line.find('choix'):]
       else :
         x = normalize(line)
         y=''
       dict_temp={x:y}
       lista.append(dict_temp) 

   dictt[i]=lista
   return dictt

#************************ Procedure ********************************

def Procedure(ch):
   lista=[]
   i="Procedure Passage"
   dictt = { i :[]}
   
   
   listax= ch.splitlines( )
   for line in listax:
          if re.search(r'«', line):
             x = normalize(line[:line.find('«')])
             y = accentfree(line[line.find('«')+1:line.rfind('»')].replace('\u00a0',''))
          elif re.search(r'bouton', line):
             x = normalize(line[line.find('bouton'):line.find('bouton')+len('bouton'):])
             y = accentfree(line[line.find('bouton')+len('bouton'):-1].replace('\u00a0',''))
          elif re.search(r'Valider', line):
             x = normalize(line[line.find('Valider'):line.find('Valider')+len('Valider'):])
             y = accentfree(line[line.find('Valider')+len('Valider'):].replace('\u2019',''))
          elif re.search(r'LP', line):
             x = normalize(line[:line.find('LP')])
             y = accentfree(line[line.find('LP'):-1].replace('\u00a0',''))
          else :
            x = normalize(line)
            y=''
          dict_temp={x:y}
          lista.append(dict_temp) 
          
   dictt[i]=lista
   return dictt
#*********************Predicted results********************************
def Results (ch) : 
   results={} 
   lista=[]
   dictt={}
   i=j=o=k=0
   m="Predicted Results"
   dictt = { m :[]}
   listax= ch.splitlines( )
   for line in listax :
       if re.search(r'«', line) and re.search(r'boutons', line) :
          x = normalize(line[:line.find('«')])
          y = accentfree(line[line.find('«')+1:line.rfind('»')].replace('\u00a0',''))  
          a = normalize(line[line.find('boutons'):line.find('boutons')+len('boutons'):])
          b = accentfree(remove_stopw(line[line.find('boutons')+len('boutons suivants :'):-2].replace(',','')).strip())
          c = b.split(' ')
          dict_temp={x:y}
          dict_temp2={a:c}
          lista.append(dict_temp)
          lista.append(dict_temp2)
       elif len(re.findall(r'«', line))==2 :
          k=line[:line.rfind('et')]
          x = normalize(k[:k.find('«')])
          y = accentfree(k[k.find('«')+1:k.rfind('»')].replace('\u00a0','')) 
          s=line[line.rfind('et'):]
          a = normalize(s[:s.find('«')])
          b = accentfree(s[s.find('«')+1:s.rfind('»')].replace('\u00a0',''))
          dict_temp={x:y}
          dict_temp2={a:b}
          lista.append(dict_temp)
          lista.append(dict_temp2)
       elif re.search(r'«', line):
          key = normalize(line[:line.find('«')])
          results[key] = accentfree(line[line.find('«')+1:line.rfind('»')].replace('\u00a0',''))
       elif re.search(r'Valider', line):
          key = normalize(l[l.find('Valider'):l.find('Valider')+len('Valider'):])
          results[key] = accentfree(l[l.find('Valider')+len('Valider'):].replace('\u2019',''))
       elif re.search(r'l’écran', line) and re.search(r'choix', line):
           key = normalize(line[:line.find('choix')])
           results[key] = accentfree(line[line.find('choix'):].replace('\u2019',''))
       else :
         key = normalize(line)
         results[key]=''
   if len(results.keys())>0 :
    lista.append(results)
   dictt[m]=lista    
   return dictt

  
#********************Prog Principale************************************

if __name__ == '__main__':

 document = Document('/content/input.docx')
 tables = []
 columns =['Initial conditions', 'Operating procedure', 'Predicted result', 'R', 'Kx', 'Comments']
 init_df= pd.DataFrame(columns=columns)
 for index,table in enumerate(document.tables): #index : numero de table 
     df = [['' for i in range(len(table.columns))] for j in range(len(table.rows))]
     for i, row in enumerate(table.rows):
         for j, cell in enumerate(row.cells):       #chaque tableau on le crée un Dataframe
             df[i][j] = cell.text
     if pd.DataFrame(df).iloc[0][0] == "Initial conditions" :  #on garde seulement les tableaux de validation
       for E in df: del E[3]
       init_df = pd.concat([init_df,pd.DataFrame(df)])


 pd.DataFrame(init_df).to_excel("Table_final.xlsx",index=False,header=None)
 dff= pd.read_excel('Table_final.xlsx')


# ///////////////////////////////////////////////
 demo = ["Test ID : 1601","Test ID : 1608","Test ID : 1611","Test ID : 1623","Test ID : 1625","Test ID : 1857","Test ID : 1660"]
 ch="Test ID :"
 globall= { i :[]}
 
 for k in range (len(dff)):
   if (ch in dff.iloc[k]['Initial conditions']):   # si on est ds la case d'ID
     if (dff.iloc[k]['Initial conditions'] in demo): # Lire uniquement les tests de la liste demo (enlever pour lire tous les ID)
       concatt=[]
       globall.clear()
       i=dff.iloc[k]['Initial conditions']  #Test ID
       res1 = Initcond(dff.iloc[k+1]['Initial conditions'])
       res2=Procedure(dff.iloc[k+1]['Operating procedure'])  
       res3=Results(dff.iloc[k+1]['Predicted result'])
       concatt.append(res1)
       concatt.append(res2)
       concatt.append(res3)
       globall[i]=concatt
       with open(str(i[10:])+".json", 'w') as f:
          json.dump(globall, f,indent=True)






 

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
